In [30]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/lesioned_sim.py'>

In [32]:
importlib.reload(main_sim_tensor)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.001, "grid_epsilon_end": 1.8, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 0,  
    "V_prior": 3, 
    "alpha_prior": 1, 
    "beta_prior": 2, 
    "epsilon": 0.001, "mu_epsilon": 0.001, "sd_epsilon": 4, 
    "hypothetical_obs_grid_n": 10, 
    "world_EIGs": 0.0001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(1)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

print(tensor_stimuli)
params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= 3)
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)


[<granch_utils.init_model_tensor.granch_stimuli object at 0x107f53520>, <granch_utils.init_model_tensor.granch_stimuli object at 0x107f53580>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2ba290a60>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2ba204550>, <granch_utils.init_model_tensor.granch_stimuli object at 0x2ba1bae90>, <granch_utils.init_model_tensor.granch_stimuli object at 0x28c90a7d0>, <granch_utils.init_model_tensor.granch_stimuli object at 0x28c5c8940>, <granch_utils.init_model_tensor.granch_stimuli object at 0x28c90ab30>]


/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/init_model_tensor.py:279: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.189' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()


    stimulus_id       EIG Look_away
0             0  1.224148     False
1             0  0.875481     False
2             0   0.48973      True
3             1  0.411448     False
4             1  0.224108     False
..          ...       ...       ...
495         NaN       NaN       NaN
496         NaN       NaN       NaN
497         NaN       NaN       NaN
498         NaN       NaN       NaN
499         NaN       NaN       NaN

[500 rows x 3 columns]
